# CAPA 2

In [1]:
# -*- coding: utf-8 -*-
from dataclasses import dataclass, field
from typing import Literal, Optional, Tuple
import math
import logging

# Etiquetas de fase fenológica
Fase = Literal['Floracion', 'Cuajado', 'PostEnvero', 'PostCosecha']

# ---------------------------------------------------------------------
# Parámetros operativos del sistema de riego
# ---------------------------------------------------------------------

@dataclass
class Params:
    """
    Parámetros de operación de la Capa 2.
    Definen restricciones hidráulicas, ventanas temporales, umbrales de decisión
    y políticas de ajuste por lluvia.
    """
    # Hidráulica y ventana temporal
    infil_mm_h: float = 0.9  # Tasa de infiltración efectiva del sistema (mm/h)
    horas_vent: float = 8.0  # Ventana de riego disponible por jornada (h), típicamente nocturna
    mm_min: float = 2.0  # Evento mínimo para evitar micropulsos ineficaces (mm)
    max_horas_pulso: float = 4.0  # Duración máxima de pulso continuo sin descanso (h)
    descanso_entre_pulsos_h: float = 0.5  # Tiempo de descanso entre pulsos (h)
    
    # Bucket de deuda (tope operativo)
    Dmax: float = 6.0  # Tope de acumulación de deuda para evitar valores irreales (mm)
    
    # Seguro temporal por días sin riego
    usar_seguro_dias: bool = False  # Activar gatillo temporal además del de deuda
    n_media: int = 5  # Ventana móvil para calcular receta media (días)
    rmedia_floor: float = 0.5  # Suelo mínimo para receta media (mm/día)
    
    # Árbol de decisión por lluvia prevista
    beta_24_posponer: float = 0.75  # Umbral de cobertura por Pe 0-24h para posponer
    confianza_pred_0_24h: float = 0.9  # Factor de confianza en previsión 0-24h
    beta_48_posponer: float = 1.0  # Umbral de cobertura combinada 0-48h
    confianza_pred_24_48h: float = 0.7  # Factor de confianza en previsión 24-48h
    
    # Ajuste por lluvia real (separado de receta)
    raw_mm: Optional[float] = None  # RAW desde Capa 0 (mm), umbral para reset completo
    lluvia_intensa_factor: float = 1.5  # Pe_real ≥ 1.5·ETc → recorte proporcional
    lluvia_muy_intensa_factor: float = 2.0  # Pe_real ≥ 2.0·ETc → recorte fuerte por exceso
    alpha_proporcional: float = 0.6  # Proporción del exceso (Pe-ETc) a descontar
    reset_dias_por_lluvia: bool = True  # Considerar lluvia intensa como pseudo-riego
    umbral_pseudo_riego_mm: Optional[float] = None  # Umbral para pseudo-riego (None → mm_min)
    
    # Política de reset/capado de deuda por buen estado hídrico
    reset_deuda: bool = True  # Activar política de reset cuando Ks > 0.9
    cap_deuda_mm: Optional[float] = 0.0  # None=sin modificar; 0.0=reset total; >0=cap suave

# ---------------------------------------------------------------------
# Estado persistente por sector
# ---------------------------------------------------------------------

@dataclass
class Estado:
    """
    Estado operativo del sector. Mantiene memoria entre días para continuidad del control.
    """
    deuda_mm: float = 0.0  # Deuda acumulada de agua pendiente de aplicar (mm)
    dias_sin_riego: int = 0  # Contador de días consecutivos sin evento de riego
    recetas_hist: list[float] = field(default_factory=list)  # Historial de recetas para r_media
    last_fase: Optional[Fase] = None  # Última fase fenológica registrada (trazabilidad)

# ---------------------------------------------------------------------
# Gatillo adaptativo de disparo
# ---------------------------------------------------------------------

def gatillo_simple(prio: float, mm_min: float, Dmax: float) -> float:
    """
    Traduce prioridad difusa [0-1] en umbral de deuda para disparo de riego.
    
    Interpolación lineal:
      - prio=0 (baja urgencia) → Dgat = Dmax (tolerar más deuda)
      - prio=1 (alta urgencia) → Dgat = mm_min (disparar con poca deuda)
    
    Esta función separa la urgencia estratégica (Capa 1) de la decisión operativa (Capa 2).
    """
    prio = max(0.0, min(1.0, prio))  # Saturar a [0,1]
    return Dmax - prio * (Dmax - mm_min)

# ---------------------------------------------------------------------
# Ajuste de deuda por lluvia REAL
# ---------------------------------------------------------------------

def ajustar_deuda_por_lluvia_real(
    deuda_actual: float,
    etc: float,
    pe_real: float,
    p: Params
) -> Tuple[float, dict]:
    """
    Ajusta la deuda acumulada en función de la lluvia efectiva REAL del día.
    Este ajuste es INDEPENDIENTE de la receta, evitando contar la lluvia dos veces.
    
    Estrategia por niveles (de más restrictivo a menos):
      1. Pe ≥ RAW → reset completo (recarga significativa)
      2. Pe ≥ 2·ETc → recorte fuerte por exceso (lluvia muy intensa)
      3. Pe ≥ ETc → recorte proporcional (lluvia significativa)
      4. Pe < ETc → sin ajuste
    
    Retorna:
        (deuda_nueva, metadatos) donde metadatos incluye tipo de ajuste y si cuenta como pseudo-riego
    """
    meta = {'tipo': None, 'reduccion_mm': 0.0, 'pseudo_riego': False}
    
    if pe_real <= 0:
        return deuda_actual, meta
    
    # Nivel 1: reset completo si Pe ≥ RAW (recarga plena del perfil)
    if p.raw_mm is not None and pe_real >= p.raw_mm:
        meta['tipo'] = 'reset_completo_raw'
        meta['reduccion_mm'] = float(deuda_actual)
        meta['pseudo_riego'] = p.reset_dias_por_lluvia
        return 0.0, meta
    
    # Nivel 2: lluvia muy intensa (Pe ≥ 2·ETc), descuenta exceso directo
    if etc > 0 and pe_real >= p.lluvia_muy_intensa_factor * etc:
        exceso = max(pe_real - etc, 0.0)
        reduccion = min(deuda_actual, exceso)
        nueva = max(0.0, deuda_actual - reduccion)
        meta['tipo'] = 'recorte_fuerte_exceso'
        meta['reduccion_mm'] = float(reduccion)
        umbral = p.umbral_pseudo_riego_mm or p.mm_min
        meta['pseudo_riego'] = p.reset_dias_por_lluvia and (reduccion >= umbral)
        return nueva, meta
    
    # Nivel 3: lluvia significativa (ETc ≤ Pe < 2·ETc), descuenta proporción del exceso
    if etc > 0 and pe_real >= etc:
        exceso = max(pe_real - etc, 0.0)
        reduccion = min(deuda_actual, p.alpha_proporcional * exceso)
        nueva = max(0.0, deuda_actual - reduccion)
        meta['tipo'] = 'recorte_proporcional'
        meta['reduccion_mm'] = float(reduccion)
        umbral = p.umbral_pseudo_riego_mm or p.mm_min
        meta['pseudo_riego'] = p.reset_dias_por_lluvia and (reduccion >= umbral)
        return nueva, meta
    
    # Nivel 4: lluvia insuficiente, sin ajuste
    meta['tipo'] = 'sin_ajuste'
    return deuda_actual, meta
# ---------------------------------------------------------------------
# Árbol de decisión por lluvia PREVISTA
# ---------------------------------------------------------------------

def arbol_decision_lluvia(
    fase: Fase,
    prioridad: float,
    deuda_actual: float,
    capacidad_riego: float,
    pe_prevista_0_24h: float,
    pe_prevista_24_48h: float,
    p: Params
) -> Tuple[str, float]:
    """
    Decide tipo de riego según previsión meteorológica.
    
    Lógica:
      1. Si Pe 0-24h ponderada cubre umbral → POSPONER
      2. Si fase crítica (Floracion/Cuajado) + prioridad alta:
         - Proteger con riego parcial ajustado por previsión
      3. Si Pe combinada 0-48h cubre umbral → POSPONER
      4. Caso contrario → riego parcial no crítico
    
    Retorna:
        (decision, mm_evento)
    """
    # Umbrales adaptativos según deuda y capacidad
    umbral_posponer_24 = p.beta_24_posponer * min(deuda_actual, capacidad_riego)
    umbral_posponer_48 = p.beta_48_posponer * min(deuda_actual, capacidad_riego)
    
    # Regla 1: posponer si previsión 0-24h es suficiente
    if pe_prevista_0_24h * p.confianza_pred_0_24h > umbral_posponer_24:
        return 'POSPONER_LLUVIA', 0.0
    
    # Regla 2: proteger fases críticas con prioridad alta
    if (fase in ('Floracion', 'Cuajado')) and (prioridad > 0.66):
        deuda_ajustada = deuda_actual - (pe_prevista_0_24h * p.confianza_pred_0_24h)
        
        # Si también hay previsión significativa 24-48h, regar solo la mitad
        if (pe_prevista_24_48h * p.confianza_pred_24_48h) > umbral_posponer_48:
            mm_evento = max(0.0, min(deuda_ajustada / 2.0, capacidad_riego))
            return ('POSPONER_LLUVIA', 0.0) if mm_evento <= 0 else ('REGAR_PARCIAL_CRITICO_MITAD', mm_evento)
        
        # Sin previsión 24-48h significativa, regar la deuda ajustada completa
        mm_evento = max(0.0, min(deuda_ajustada, capacidad_riego))
        return ('POSPONER_LLUVIA', 0.0) if mm_evento <= 0 else ('REGAR_PARCIAL_CRITICO', mm_evento)
    
    # Regla 3: evaluar previsión combinada 0-48h
    if (pe_prevista_0_24h * p.confianza_pred_0_24h +
        pe_prevista_24_48h * p.confianza_pred_24_48h) > umbral_posponer_48:
        return 'POSPONER_LLUVIA', 0.0
    
    # Regla 4: riego parcial ajustando por previsión combinada
    deuda_ajustada = deuda_actual - (
        pe_prevista_0_24h * p.confianza_pred_0_24h +
        pe_prevista_24_48h * p.confianza_pred_24_48h
    )
    mm_evento = max(0.0, min(deuda_ajustada, capacidad_riego))
    return ('POSPONER_LLUVIA', 0.0) if mm_evento <= 0 else ('REGAR_PARCIAL_NO_CRITICO', mm_evento)

# ---------------------------------------------------------------------
# Planificación operativa de pulsos
# ---------------------------------------------------------------------

def planificar_pulsos(mm_evento: float, p: Params) -> Optional[dict]:
    """
    Divide el evento en N pulsos si la duración continua excede max_horas_pulso.
    
    Restricciones:
      - Cada pulso no supera max_horas_pulso de riego continuo
      - Entre pulsos hay descanso_entre_pulsos_h
      - Si con descansos no cabe en horas_vent, recorta mm_evento para ajustar
    
    Retorna:
        dict con plan operativo o None si no requiere división
    """
    if mm_evento <= 0:
        return None
    
    horas_cont = mm_evento / p.infil_mm_h
    
    # Si cabe en un pulso sin dividir
    if horas_cont <= p.max_horas_pulso:
        return {
            'num_pulsos': 1,
            'mm_por_pulso': round(mm_evento, 2),
            'horas_por_pulso': round(horas_cont, 2),
            'descanso_entre_pulsos_h': 0.0,
            'horas_totales_aprox': round(horas_cont, 2)
        }
    
    # Calcular número mínimo de pulsos para no exceder max_horas_pulso
    N = max(2, math.ceil(horas_cont / p.max_horas_pulso))
    horas_totales = horas_cont + (N - 1) * p.descanso_entre_pulsos_h
    
    # Si no cabe en la ventana, recortar mm_evento proporcionalmente
    if horas_totales > p.horas_vent:
        horas_riego_disponibles = p.horas_vent - (N - 1) * p.descanso_entre_pulsos_h
        if horas_riego_disponibles <= 0:
            return None  # Sin tiempo útil disponible
        mm_fit = p.infil_mm_h * horas_riego_disponibles
        mm_evento = min(mm_evento, mm_fit)
        horas_cont = mm_evento / p.infil_mm_h
    
    # Distribución uniforme entre pulsos
    mm_por_pulso = mm_evento / N
    horas_por_pulso = mm_por_pulso / p.infil_mm_h
    horas_totales_aprox = N * horas_por_pulso + (N - 1) * p.descanso_entre_pulsos_h
    
    return {
        'num_pulsos': N,
        'mm_por_pulso': round(mm_por_pulso, 2),
        'horas_por_pulso': round(horas_por_pulso, 2),
        'descanso_entre_pulsos_h': p.descanso_entre_pulsos_h,
        'horas_totales_aprox': round(horas_totales_aprox, 2)
    }

# ---------------------------------------------------------------------
# Planificador diario (núcleo de Capa 2)
# ---------------------------------------------------------------------

def paso_diario(
    fase: Fase,
    Ks: float,
    etc: float,
    pe_real: float,
    pct_dnd: float,
    prioridad: float,
    pe_prevista_0_24h: float,
    pe_prevista_24_48h: float,
    estado: Estado,
    p: Params
) -> dict:
    """
    Planificador diario de Capa 2: del criterio a la acción.
    
    Secuencia:
      1. Calcular receta del día y acumular en bucket de deuda
      2. Aplicar reset/capado si Ks > 0.9 en fases no críticas
      3. Ajustar deuda por lluvia REAL del día
      4. Traducir prioridad a umbral de gatillo (Dgat)
      5. Decidir si disparar riego por deuda o por días
      6. Si dispara: aplicar árbol de previsión
      7. Planificar pulsos y actualizar estado
    
    Parámetros:
        fase: fase fenológica actual
        Ks: coeficiente de estrés hídrico [0-1]
        etc: evapotranspiración del cultivo (mm)
        pe_real: precipitación efectiva real del día (mm)
        pct_dnd: porcentaje de DND a reponer (%)
        prioridad: prioridad difusa [0-1]
        pe_prevista_0_24h: Pe prevista 0-24h (mm)
        pe_prevista_24_48h: Pe prevista 24-48h (mm)
        estado: Estado del sector
        p: Params de operación
    
    Retorna:
        dict con decisión, mm_evento, plan_pulsos, deuda actualizada, etc.
    """
    # Inicializar variables de salida
    motivo_posponer = None
    plan_pulsos = None
    mm_evento = 0.0
    
    # 1) Receta del día: acumular en bucket de deuda
    dnd = max(etc - pe_real, 0.0)  # Demanda neta diaria (no negativa)
    receta_dia = max((pct_dnd / 100.0) * dnd, 0.0)  # Fracción de DND a reponer
    estado.deuda_mm = min(p.Dmax, max(0.0, estado.deuda_mm + receta_dia))  # Saturar a [0, Dmax]
    
    # 1b) Reset/capado por buen estado hídrico (Ks > 0.9 en fases no críticas)
    # Útil para inducir estrés tras floración en estrategia de calidad
    if p.reset_deuda and (fase in ('Cuajado', 'PostEnvero', 'PostCosecha')) and (Ks is not None) and (Ks > 0.9):
        if p.cap_deuda_mm is None:
            pass  # No modificar deuda
        elif p.cap_deuda_mm <= 0.0:
            estado.deuda_mm = 0.0  # Reset total
        else:
            estado.deuda_mm = min(estado.deuda_mm, p.cap_deuda_mm)  # Cap suave
        estado.dias_sin_riego = 0
        estado.recetas_hist.clear()
    
    # 1c) Ajuste por lluvia REAL (separado de receta para trazabilidad)
    deuda_ajustada, meta_lluvia_real = ajustar_deuda_por_lluvia_real(
        deuda_actual=estado.deuda_mm,
        etc=etc,
        pe_real=pe_real,
        p=p
    )
    estado.deuda_mm = deuda_ajustada
    
    # Si la lluvia cuenta como pseudo-riego, resetear contador de días
    if meta_lluvia_real.get('pseudo_riego', False) and p.reset_dias_por_lluvia:
        estado.dias_sin_riego = 0
    
    # 2) Gatillo adaptativo desde prioridad
    Dgat = gatillo_simple(prioridad, p.mm_min, p.Dmax)
    
    # 2b) Seguro temporal (opcional): gatillo por días sin regar
    if p.usar_seguro_dias:
        estado.recetas_hist.append(receta_dia)
        if len(estado.recetas_hist) > p.n_media:
            estado.recetas_hist.pop(0)
        r_media = max(sum(estado.recetas_hist) / len(estado.recetas_hist), p.rmedia_floor)
        S_obj = max(1, math.ceil(Dgat / r_media))  # Días objetivo para disparo
    else:
        r_media = None
        S_obj = None
    
    # 3) Decisión de disparo: por deuda o por días
    disparo_por_deuda = (estado.deuda_mm >= Dgat)
    disparo_por_dias = (p.usar_seguro_dias and estado.dias_sin_riego >= S_obj)
    dispara_gatillo = (disparo_por_deuda or disparo_por_dias)
    
    if not dispara_gatillo:
        # No se cumple umbral de disparo
        decision = 'POSPONER_DEUDA'
        mm_evento = 0.0
        motivo_posponer = 'deuda_insuficiente'
        estado.dias_sin_riego += 1
    else:
        # 4) Gatillo activado: evaluar previsión de lluvia
        cap_evento = p.infil_mm_h * p.horas_vent  # Capacidad máxima por jornada
        hay_pronostico = (max(pe_prevista_0_24h, 0.0) + max(pe_prevista_24_48h, 0.0)) > 0.0
        
        if not hay_pronostico:
            # Sin previsión: riego completo hasta capacidad
            mm_evento = min(estado.deuda_mm, cap_evento)
            decision = 'REGAR_COMPLETO'
        else:
            # Con previsión: aplicar árbol de decisión
            decision_lluvia, mm_evento = arbol_decision_lluvia(
                fase, prioridad, estado.deuda_mm, cap_evento,
                pe_prevista_0_24h, pe_prevista_24_48h, p
            )
            
            if decision_lluvia == 'POSPONER_LLUVIA':
                decision = 'POSPONER_LLUVIA'
                mm_evento = 0.0
                motivo_posponer = 'prevision_lluvia'
                estado.dias_sin_riego += 1
            else:
                decision = decision_lluvia
        
        # 5) Ejecución operativa si no se pospone
        if not decision.startswith('POSPONER'):
            # Validar evento mínimo
            if mm_evento < p.mm_min:
                decision = 'POSPONER_OPERATIVO'
                mm_evento = 0.0
                motivo_posponer = 'evento_menor_a_minimo'
                estado.dias_sin_riego += 1
            else:
                # Planificar pulsos
                plan = planificar_pulsos(mm_evento, p)
                if plan is not None:
                    plan_pulsos = plan
                    mm_evento = plan['mm_por_pulso'] * plan['num_pulsos']  # Ajuste por recorte
                    estado.deuda_mm = max(0.0, estado.deuda_mm - mm_evento)  # Descontar de deuda
                    estado.dias_sin_riego = 0  # Reset contador
                else:
                    # No cabe en ventana
                    decision = 'POSPONER_OPERATIVO'
                    mm_evento = 0.0
                    motivo_posponer = 'no_cabe_en_ventana'
                    estado.dias_sin_riego += 1
    
    # 6) Salida con trazabilidad completa
    out = {
        'fase': fase,
        'prio': round(prioridad, 3),
        'receta_dia': round(receta_dia, 2),
        'ajuste_lluvia_real': {
            'tipo': meta_lluvia_real['tipo'],
            'reduccion_mm': round(meta_lluvia_real['reduccion_mm'], 2),
            'pseudo_riego': meta_lluvia_real['pseudo_riego'],
        },
        'Dgat_mm': round(Dgat, 2),
        'decision': decision,
        'motivo_posponer': motivo_posponer,
        'mm_evento': round(mm_evento, 2),
        'deuda': round(estado.deuda_mm, 2),
        'dias': estado.dias_sin_riego,
        'plan_pulsos': plan_pulsos,
        'S_obj': (S_obj if p.usar_seguro_dias else None),
        'r_media': (round(r_media, 2) if (p.usar_seguro_dias and r_media is not None) else None)
    }
    
    # Actualizar memoria de fase
    estado.last_fase = fase
    
    return out

# EJEMPLO DE USO

In [3]:
if __name__ == "__main__":
    import logging, pandas as pd
    logging.basicConfig(level=logging.INFO)

    # --- Parámetros y estado ---
    p = Params(
        infil_mm_h=0.9,
        horas_vent=8.0,
        mm_min=2,
        max_horas_pulso=4.0,
        descanso_entre_pulsos_h=0.5,
        Dmax=10.0,
        # Ajuste lluvia REAL
        raw_mm=32.0,
        lluvia_intensa_factor=1.5,
        lluvia_muy_intensa_factor=2.0,
        alpha_proporcional=0.6,
        reset_dias_por_lluvia=True,
        # >>> clave para evitar el "cero total":
        reset_deuda=True,
        cap_deuda_mm=None   # None = no reset/cap por Ks>0.9
    )
    estado = Estado()

    # (día, fase, ETc, Pe_real, %DND, prioridad, Pe0-24, Pe24-48)
    dias = [
        ("D1", "Cuajado", 3.128, 0.0, 52.34, 0.605, 0.0, 0.0),
        ("D2", "Cuajado", 3.353, 0.0, 63.28, 0.725, 0.0, 0.0),
        ("D3", "Cuajado", 3.895, 0.0, 58.42, 0.423, 0.0, 0.0),
        ("D4", "Cuajado", 4.126, 0.0, 71.64, 0.575, 3.52, 1.3),
        ("D5", "Cuajado", 3.462, 3.2, 62.32, 0.587, 2.4, 0.0),
        ("D6", "Cuajado", 3.755, 2.3, 68.83, 0.512, 3.1, 0.0),
        ("D7", "Cuajado", 4.321, 0.0, 62.38, 0.634, 0.0, 0.0),
    ]

    filas = []
    for dia, fase, etc, pe_real, pct_dnd, prio, pe024, pe2448 in dias:
        out = paso_diario(
            fase=fase, Ks=1.0,  # si prefieres aplicar la política, pon Ks=0.85 los días normales
            etc=etc, pe_real=pe_real,
            pct_dnd=pct_dnd, prioridad=prio,
            pe_prevista_0_24h=pe024, pe_prevista_24_48h=pe2448,
            estado=estado, p=p
        )
        filas.append({
            "Día": dia, "Fase": fase,
            "ETc": round(etc, 1), "PeR": round(pe_real, 1),
            "%DND": int(round(pct_dnd)), "Prio": round(prio, 2),
            "Pe0-24": round(pe024, 1), "Pe24-48": round(pe2448, 1),
            "Receta": out["receta_dia"], "Adj_LL": out["ajuste_lluvia_real"]["reduccion_mm"],
            "D_gat": out["Dgat_mm"], "Decisión": out["decision"],
            "mm_evt": out["mm_evento"], "Deuda": out["deuda"], "Días": out["dias"],
        })

    df = pd.DataFrame(filas, columns=[
        "Día","Fase","ETc","PeR","%DND","Prio","Pe0-24","Pe24-48",
        "Receta","Adj_LL","D_gat","Decisión","mm_evt","Deuda","Días"
    ])
    print(df.to_string(index=False))


Día    Fase  ETc  PeR  %DND  Prio  Pe0-24  Pe24-48  Receta  Adj_LL  D_gat                 Decisión  mm_evt  Deuda  Días
 D1 Cuajado  3.1  0.0    52  0.60     0.0      0.0    1.64     0.0   5.16           POSPONER_DEUDA    0.00   1.64     1
 D2 Cuajado  3.4  0.0    63  0.72     0.0      0.0    2.12     0.0   4.20           POSPONER_DEUDA    0.00   3.76     1
 D3 Cuajado  3.9  0.0    58  0.42     0.0      0.0    2.28     0.0   6.62           POSPONER_DEUDA    0.00   6.03     1
 D4 Cuajado  4.1  0.0    72  0.57     3.5      1.3    2.96     0.0   5.40 REGAR_PARCIAL_NO_CRITICO    4.92   4.07     0
 D5 Cuajado  3.5  3.2    62  0.59     2.4      0.0    0.16     0.0   5.30           POSPONER_DEUDA    0.00   4.23     1
 D6 Cuajado  3.8  2.3    69  0.51     3.1      0.0    1.00     0.0   5.90           POSPONER_DEUDA    0.00   5.24     1
 D7 Cuajado  4.3  0.0    62  0.63     0.0      0.0    2.70     0.0   4.93           REGAR_COMPLETO    6.76   1.17     0
